for raw data

In [1]:
import json
with open('data2.json','r', encoding="utf-8") as f:
    raw= json.load(f)

In [2]:
print(type(raw))
print(list(raw.keys())[:5])

<class 'dict'>
['0', '1', '2', '3', '4']


for washed data

In [4]:
with open('wash1.json','r', encoding="utf-8") as f:
    wash= json.load(f)

In [5]:
set_keyword = set()
for item in wash:
    for each_explain in wash[item]:
        temp_set = set(list(each_explain.keys()))
        set_keyword = set_keyword | temp_set
set_keyword

{'定义', '定义来源', '术语来源', '缩写'}

wash process

In [3]:
import string
def if_sup_sub(str, sub_str):
    """
    premise:
        如果这个带上下标的word在最后，或者后面是逗号句号空格
        标记为@@@
    """
    """
    length = len(str)
    find_all_ls = []
    result = re.finditer(pat,s)
    for i in result:
        find_all_ls.append(i.span())
    # find_all_ls: [(7, 13),(25, 31)]

    find_all_ls_filter = []
    for idx in find_all_ls:
        # 判断是不是句子的最后
        if idx[-1] == length:
            find_all_ls_filter.append(idx)
        # 判断句子后面是不是符号
        if str[idx[-1]] in string.punctuation:
            find_all_ls_filter.append(idx)

    find_all_ls_filter = sorted(find_all_ls_filter)

    if len(find_all_ls_filter) >= 1:
        return find_all_ls_filter
    else:
        return False # 没找到
    """
    ori_str = str[:]
    str = str.replace(sub_str+'.',"@@@"+sub_str+'.')
    str = str.replace(sub_str+';',"@@@"+sub_str+';')
    str = str.replace(sub_str+',',"@@@"+sub_str+',')
    str = str.replace(sub_str+' ',"@@@"+sub_str+' ')
    str = str.replace(sub_str+'"',"@@@"+sub_str+'"')
    str = str.replace(sub_str+':',"@@@"+sub_str+':')
    
    if str[-len(sub_str):] == sub_str:
        str = str[:-len(sub_str)] + "@@@" + str[-len(sub_str):]

    if ori_str != str: # 如果改变
        return str
        
    else:
        if len(sub_str) == 1:
            return False # 如果substr长度=1，并且不出现在末尾，那么不改变
        return True # 直接replace改


def replace_with_sup_sub(str, sub_Str, sup = True):
    if sup == True:
        sup_sub = "sup"
    else:
        sup_sub = "sub"

    str_sign = if_sup_sub(str, sub_Str)
    if str_sign == False:
        return str
    if str_sign == True:
        return str.replace(sub_Str, "(" + sup_sub + " " + sub_Str + ")")
        


    res_str = str_sign.replace("@@@" + sub_Str, "(" + sup_sub + " " + sub_Str + ")")

    res_str = res_str.replace("@@@", "")
    
    return res_str

In [15]:
def seperate_type(a):

    if type(a) == str:
        if a[0] == 'I':
            return "术语来源"
        else:
            print("ERROR1")
            # print(a)
    
    with_link = False
    if type(a) == list:
        if len(a) == 4:
            with_link = True

        if "SOURCE: " in a[0]:
            return "定义来源", with_link
        elif a[0].isupper():
            return "缩写", with_link
        else:
            return "定义", with_link
    else:
        print("ERROR1")

def handle_dingyi_source(str):
    str = str.replace("[SOURCE: ", "")
    str = str.replace("SOURCE: ", "")

    str = str.replace("]", "")
    return str

In [8]:
import copy
def handle_each_ls_from_sub_sup(explain):
    res = explain[0]

    if len(explain[1]) != 0:
        for subitem in explain[1]: # sup
            res = replace_with_sup_sub(res, subitem, sup=True)
    
    if len(explain[2]) != 0:
        for subitem in explain[2]: # sub
            res = replace_with_sup_sub(res, subitem, sup=False)
            
    return res

def detect_none(s):
    if s == None:
        return True
    if type(s) == list:
        if s[0] == None:
            return True

In [22]:
dic = {}

# initualization
for each_page in raw:
    for each_term in raw[each_page]:
        dic[each_term] = []

error = 0

for each_page in raw:
    for each_term in raw[each_page]:
        
        term_dic = {}

        for each_explain in raw[each_page][each_term]:
            # print(each_explain)
            if detect_none(each_explain): 
                error+= 1
                continue
            
            explain_type = seperate_type(each_explain)
            
            if explain_type == "术语来源":
                term_dic["术语来源"] = each_explain

            if explain_type[-1] == True:
                term_dic["图片地址"] = each_explain[-1]

            if explain_type[0] == "定义来源":
                dingyi_source = handle_dingyi_source(each_explain[0])
                term_dic["定义来源"] = dingyi_source
            
            if explain_type[0] == "定义":
                dingyi = handle_each_ls_from_sub_sup(each_explain)

                if "定义" in term_dic:
                    term_dic["定义"].append(dingyi)
                else:
                    term_dic["定义"] = [dingyi]

            if explain_type[0] == "缩写":
                suoxie = handle_each_ls_from_sub_sup(each_explain)
                term_dic["缩写"] = suoxie
            
            
        dic[each_term].append(term_dic)

In [23]:
error

1

In [10]:
def connect_Str(ls):
    str_res = ""
    for i in ls:
        str_res += i
    return str_res

In [24]:
# 去除重合
dic_filter = {}

# initualization
for each_page in raw:
    for each_term in raw[each_page]:
        dic_filter[each_term] = []

cnt = 0
for item in dic:
    if len(dic[item]) > 1:

        set_definition = set()
        
        for i in range(len(dic[item])):
            if "定义" not in dic[item][i]:
                # print(dic[item][i])
                # print("--"*20)
                cnt += 1
                dic_filter[item].append(dic[item][i])
                continue
            if connect_Str(dic[item][i]["定义"]) not in set_definition:
                set_definition.add(connect_Str(dic[item][i]["定义"]))
                dic_filter[item].append(dic[item][i])

    else:
        dic_filter[item] = dic[item]




In [25]:
cnt

899

In [26]:
with open('wash_2_select.json','w',encoding='utf-8') as file_obj:
    json.dump(dic_filter,file_obj,ensure_ascii=False,indent = 4)